In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

In [2]:
data = spark.read.csv('hack_data.csv', header = True, inferSchema = True)

In [4]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [6]:
feature_col = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [5]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=feature_col,
                           outputCol='feature')

In [8]:
final_data = assembler.transform(data)

In [9]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- feature: vector (nullable = true)



In [10]:
from pyspark.ml.feature import StandardScaler

In [11]:
scaler = StandardScaler(inputCol='feature',
                       outputCol='scaledfeatures')

In [12]:
final_data = scaler.fit(final_data).transform(final_data)

In [13]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- feature: vector (nullable = true)
 |-- scaledfeatures: vector (nullable = true)



In [14]:
km2 = KMeans(featuresCol='scaledfeatures', k = 2)
km3 = KMeans(featuresCol='scaledfeatures', k = 3)

In [15]:
kmm2 = km2.fit(final_data)
kmm3 = km3.fit(final_data)

In [17]:
kmm3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   79|
|         2|   88|
|         0|  167|
+----------+-----+



In [18]:
kmm2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

